In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper', font_scale=1.6)
import keras.backend as K

SEED = 0
SAVE_PLOTS = False

In [ ]:
import numpy as np
import pandas as pd
import glob
import itertools
import os
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, ParameterGrid

RF_PARAM_GRID = {'n_estimators': [50, 100, 250], 'criterion': ['gini', 'entropy'],
                 'max_features': [0.05, 0.1, 0.2, 0.3], 'min_samples_leaf': [1, 2, 3]}
#RF_PARAM_GRID = {'bootstrap': [True], 'class_weight': [None], 'criterion': ['entropy'], 'max_depth': [None], 'max_features': [0.3], 'max_leaf_nodes': [None], 'min_impurity_split': [1e-07], 'min_samples_leaf': [1], 'min_samples_split': [2], 'min_weight_fraction_leaf': [0.0], 'n_estimators': [250], 'n_jobs': [1], 'oob_score': [False], 'random_state': [0], 'verbose': [0], 'warm_start': [False]}

def parse_logs(log_files, max_epoch=250):
    logs = [pd.read_csv(f, index_col='epoch', parse_dates=[1]) for f in log_files]
    for log, f in zip(logs, log_files):
        run = f.split('/')[-2]
        log.drop(log.index[log.index > max_epoch], axis=0, inplace=True)
        if 'time' not in log:
            raise ValueError("Missing times from {}".format(f))
        log.columns = ['time', run + ' Train', run + ' Valid']
        log['time'] = (log['time'] - log['time'].min()) / np.timedelta64(1, 's')
    step_logs = pd.concat([l.drop('time', axis=1, inplace=False) for l in logs], axis=1)
    time_logs = pd.concat([l.set_index('time') for l in logs], axis=1)
    
    return step_logs, time_logs


def run_to_args(run):
    values = run.split(' ')[0].split('_')
    args = {'model_type': values[0].upper(), 'size': int(values[1]), 
            'num_layers': int(values[2].strip('x')),
            'drop_frac': float(values[4].strip('drop')),
            'bidirectional': 'bidir' in run}
    if 'emb' in run:
        args['embedding'] = int([v for v in values if 'emb' in v][0][3:])
    return args
    

def training_plot(logs, loss_type='Valid', ylim=(1e-2, 1e0)):
    LABEL_MAP = {'epoch': 'Epoch', 'time': 'Time (s)'}
    sns.set_style("darkgrid")
    fig = plt.figure()
    colors = sns.color_palette('deep', n_colors=int(len(logs.columns) / 2))
    for i, c in enumerate(logs.columns):
        if loss_type is not None and loss_type not in c:
            continue
        to_plot = logs.iloc[:, i]#logs[c]
        to_plot = to_plot.dropna(inplace=False)
        args = run_to_args(c)
        to_plot.name = "{model_type}({size}) x {num_layers}".format(**args)
        if 'embedding' in args:
            to_plot.name += ", Embedding={}".format(args['embedding'])
        to_plot.plot(color=colors[int(i / 2)], legend=True, logy=True)
    plt.xlabel(LABEL_MAP[logs.index.name])
    plt.ylabel('Loss')
    plt.ylim(ylim)
    return fig


def plot_confusion_matrix(y, y_pred, classnames, filename=None):
    classnames = sorted(classnames)
    sns.set_style("whitegrid", {'axes.grid' : False})
    cm = confusion_matrix(y, y_pred, classnames)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues, vmin=0,
               vmax=np.unique(y, return_counts=True)[1].max())
    #plt.title(title)
    #plt.colorbar()
    tick_marks = np.arange(len(classnames))
    plt.xticks(tick_marks, classnames, rotation=45)
    plt.yticks(tick_marks, classnames)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label');
    if filename:
        plt.savefig(filename)
        
        from copy import deepcopy

def oob_search(model, param_lists, X, y):
    best_model = None
    best_params = None
    best_score = -np.inf
    
    for g in ParameterGrid(param_lists):
        model.set_params(**g)
        model.fit(X, y)
        if model.oob_score_ > best_score:
            best_score = model.oob_score_
            best_params = g
            best_model = deepcopy(model)
            
    return best_model, best_params, best_score

## Period estimator training plots

In [ ]:
log_files = glob.glob('keras_logs/period/uneven/noise0.5/gru_*x2*_bidir/training.csv')

step_logs, time_logs = parse_logs(log_files)
step_plot = training_plot(step_logs)
time_plot = training_plot(time_logs)
if SAVE_PLOTS:
    step_plot.savefig('paper/figures/gru_period_step.pdf')
    time_plot.savefig('paper/figures/gru_period_time.pdf')

In [ ]:
log_files = (glob.glob('keras_logs/period/uneven/noise0.5/gru*64*bidir/training.csv')
             + glob.glob('keras_logs/period/uneven/noise0.5/lstm*64*bidir/training.csv'))

step_logs, time_logs = parse_logs(log_files)
step_plot = training_plot(step_logs)
time_plot = training_plot(time_logs)
if SAVE_PLOTS:
    step_plot.savefig('figures/gru_lstm_period_step.pdf')
    time_plot.savefig('figures/gru_lstm_period_time.pdf')

In [ ]:
K.set_learning_phase(0)
%run period.py --size 96 --num_layers 2 --drop_frac 0.25 --no_train --model_type gru --sigma 0.5 --lr 5e-4 --sim_type period/uneven/noise0.5 --bidirectional
scaler.inverse_transform(Y, copy=False);

In [ ]:
%%time
train = np.arange(args.N_train)
test = args.N_train + np.arange(args.N_test)
pred_gru = model.predict(X)
scaler.inverse_transform(pred_gru)

In [ ]:
%%time
import gatspy
from distutils.version import LooseVersion
assert LooseVersion(gatspy.__version__) < '0.3.0'

pred_gat = np.zeros(pred_gru.shape)
for i in range(pred_gat.shape[0]):
    t = np.cumsum(X[i, :, 0])
    x = X[i, :, 1]
    opt_args = {'period_range': (1.0, 10.0), 'quiet': True}
#    opt_args = {'period_range': (0.05, 0.95 * (t.max() - t.min())), 'quiet': True}
    model_gat = gatspy.periodic.LombScargleFast(fit_period=True, optimizer_kwds=opt_args)#, silence_warnings=True)
    model_gat.fit(t, x)
    omega = 2 * np.pi / model_gat.best_period
    off, A2, A1 = model_gat._best_params(omega)
#    pred_gat[i] = np.array([model_gat.best_period, np.sqrt(A1 ** 2 + A2 ** 2), np.arctan2(A2, A1), off + model_gat.ymean_])
    pred_gat[i] = np.array([model_gat.best_period, A1, A2, off + model_gat.ymean_])

In [ ]:
inds = test
fig = sns.jointplot(pred_gru[inds, 0] - Y[inds, 0], pred_gat[inds, 0] - Y[inds, 0], xlim=(-1., 1.), ylim=(-1., 1.), stat_func=None)
fig.ax_joint.annotate("GRU MSE: {:1.4f}\nLomb-Scargle MSE: {:1.4f}".format(np.mean((pred_gru[inds, 0] - Y[inds, 0]) ** 2), np.mean((pred_gat[inds, 0] - Y[inds, 0]) ** 2)), (-0.95, 0.78))
fig.ax_joint.set_xlabel('GRU error')
fig.ax_joint.set_ylabel('Lomb-Scargle error');
#plt.title("GRU MAE: {}\nGatspy MAE: {}".format(np.median(np.abs(pred_gru[:, 0] - Y[inds, 0])), np.median(np.abs(pred_gat[:, 0] - Y[inds, 0]))))
#sns.jointplot(pred_gru[inds, 0], Y[inds, 0])
#sns.jointplot(pred_gat[inds, 0], Y[inds, 0])
if SAVE_PLOTS:
    plt.savefig('figures/ls_period.pdf')

## Autoencoder training plots

In [ ]:
from natsort import natsorted

log_files = natsorted(glob.glob('keras_logs/autoencoder/uneven/noise0.5/gru_064_x2*/training.csv'))
log_files = [l for l in log_files if 'emb64' not in l]

step_logs, time_logs = parse_logs(log_files)
step_plot = training_plot(step_logs, ylim=(0.01, 2.0))
time_plot = training_plot(time_logs, ylim=(0.01, 2.0))
if SAVE_PLOTS:
    step_plot.savefig('paper/figures/gru_autoencoder_step.pdf')
    time_plot.savefig('paper/figures/gru_autoencoder_time.pdf')

## ASAS reconstructions

In [ ]:
%%time
import os
import joblib
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder

arg_dict = {'size': 64, 'embedding': 64, 'num_layers': 1, 'model_type': 'gru',
            'sim_type': 'asas_full/n200_ss0.7', 'n_min': 200, 'n_max': 200,
            'lr': 1e-3, 'bidirectional': True, 'ss_resid': 0.7,
            'drop_frac': 0.25, 'survey_files': ['data/asas/full.pkl'],
            'period_fold': False, 'no_train': True}
run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X, X_raw, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)
full = joblib.load('data/asas/full.pkl')
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
if args.ss_resid:
    split = [el for el in split if el.ss_resid <= args.ss_resid]

In [ ]:
(len(X), 0.8 * len(X), 0.2 * len(X))

In [ ]:
%%time
pred = model.predict({'main_input': X[:], 'aux_input': X[:, :, [0,]]})
ords = np.argsort(np.sum((pred.squeeze() - X[:, :, 1]) ** 2, axis=1))
inds, labels = zip(*[(8320, '25'), (23994, '75')])
for ord_i, label in zip(inds, labels):
    i = ords[ord_i]
    print(i)
    t = X_raw[~wrong_units][i, :, 0]
    m = X_raw[~wrong_units][i, :, 1]
    e = X_raw[~wrong_units][i, :, 2]
    m_est = pred[i] * scales[i, 0] + means[i]
    plt.figure()
    plt.errorbar(t, m, e, None, 'o');
#    plt.errorbar(t, m_est, None, None, 'o');
    plt.xlabel('Time [day]')
    plt.ylabel('Magnitude')
    plt.legend(['Original', 'Reconstructed'])
    plt.title(f'{split[i].survey} {split[i].name} ({split[i].label})')
    plt.gca().invert_yaxis()
    if SAVE_PLOTS:
        plt.savefig(f'paper/figures/asas_reconstruct_{label}.pdf')

In [ ]:
%%time
import os
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder

arg_dict = {'size': 64, 'embedding': 64, 'num_layers': 1, 'model_type': 'gru',
            'sim_type': 'asas_fold/n200', 'n_min': 200, 'n_max': 200,
            'lr': 1e-3, 'bidirectional': True, 'ss_resid': 0.7,
            'drop_frac': 0.25, 'survey_files': ['data/asas/full.pkl'],
            'period_fold': True, 'no_train': True}
run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X_fold, X_raw_fold, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)

In [ ]:
pred_fold = model.predict({'main_input': X_fold[:], 'aux_input': X_fold[:, :, [0,]]})
ords_fold = np.argsort(np.sum((pred_fold.squeeze() - X_fold[:, :, 1]) ** 2, axis=1))
for ord_i, label in zip(inds, labels):
    i = ords[ord_i]
    t = X_raw_fold[~wrong_units][i, :, 0]
    m = X_raw_fold[~wrong_units][i, :, 1]
    e = X_raw_fold[~wrong_units][i, :, 2]
    m_est = (pred[i] * scales[i, 0] + means[i])[np.argsort(split[i].times % split[i].p)]
    m_fold = pred_fold[i] * scales[i, 0] + means[i]
    plt.figure()
    plt.errorbar(t, m, e, None, 'o');
    plt.errorbar(t, m_est, None, None, 'o', alpha=0.6);
    plt.errorbar(t, m_fold, None, None, 'o');
    plt.xlabel('Phase [day]')
    plt.ylabel('Magnitude')
    plt.title(f'{split[i].survey} {split[i].name} ({split[i].label})')
    plt.legend(['Original', 'Reconstructed (non-folded)', 'Reconstructed (folded)'])
    plt.gca().invert_yaxis()
    if SAVE_PLOTS:
        plt.savefig(f'figures/asas_folded_reconstruct_{label}.pdf')

## ASAS autoencoder random forests

In [ ]:
%%time
import os
import joblib
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from light_curve import LightCurve
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder, preprocess

arg_dict = {'size': 96, 'embedding': 64, 'num_layers': 2, 'model_type': 'gru',
            'sim_type': 'asas_fold_early/n200_ss0.7', 'n_min': 200, 'n_max': 200,
            'lr': 5e-4, 'bidirectional': True, 'ss_resid': 0.7,
            'drop_frac': 0.25, 'survey_files': ['data/asas/full.pkl'],
            'period_fold': True, 'no_train': True}
run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X, X_raw, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)

top_classes = ['RR_Lyrae_FM', 'W_Ursae_Maj', 'Classical_Cepheid', 'Beta_Persei', 'Semireg_PV']
full = joblib.load('data/asas/full.pkl')
full = [lc for lc in full if lc.label in top_classes]
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
#if args.ss_resid:
#    split = [el for el in split if el.ss_resid <= args.ss_resid]
if args.period_fold:
    for lc in split:
        lc.period_fold()
X_list = [np.c_[lc.times, lc.measurements, lc.errors] for lc in split]
classnames, indices = np.unique([lc.label for lc in split], return_inverse=True)
periods = [lc.p for lc in split]
y = classnames[indices]
train, valid = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y))[0]

X_raw = pad_sequences(X_list, value=0., dtype='float', padding='post')
X, means, scales, wrong_units = preprocess(X_raw, args.m_max)
encode_model = Model(input=model.input, output=model.get_layer('encoding').output)
encoding = encode_model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)})
encoding = np.c_[encoding, means, scales, periods]

In [ ]:
len(fset)

In [ ]:
%%time
#asas_model = RandomForestClassifier(oob_score=True)
#asas_model, best_params, oob_score = oob_search(asas_model, RF_PARAM_GRID, encoding[train], y[train])
asas_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
asas_model.fit(encoding[train], y[train])
print("Training score: {}%".format(100 * asas_model.score(encoding[train], y[train])))
print("Validation score: {}%".format(100 * asas_model.score(encoding[valid], y[valid])))
plot_confusion_matrix(y[valid], asas_model.predict(encoding[valid]), classnames,
                      'figures/asas_confusion.pdf' if SAVE_PLOTS else None)

## ASAS Richards random forests

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold
from light_curve import LightCurve
import joblib
from cesium.features import LOMB_SCARGLE_FEATS, CADENCE_FEATS, GENERAL_FEATS
from cesium.featurize import featurize_time_series

from argparse import Namespace; args = Namespace(n_min=200, n_max=200)

top_classes = ['RR_Lyrae_FM', 'W_Ursae_Maj', 'Classical_Cepheid', 'Beta_Persei', 'Semireg_PV']
full = joblib.load('data/asas/full.pkl')
full = [lc for lc in full if lc.label in top_classes]
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
X_list = [np.c_[lc.times, lc.measurements, lc.errors] for lc in split]
classnames, indices = np.unique([lc.label for lc in split], return_inverse=True)
y = classnames[indices]
train, valid = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y))[0]

times, measurements, errors = zip(*[x.T for x in X_list])

features_to_use = LOMB_SCARGLE_FEATS + GENERAL_FEATS# + CADENCE_FEATS

#fset = featurize_time_series(times, measurements, errors, features_to_use)
#joblib.dump(fset, 'asas_richards.pkl', compress=True)
fset = joblib.load('asas_richards.pkl')

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

#cs_model = RandomForestClassifier(oob_score=True)
#cs_model, best_params, oob_score = oob_search(cs_model, RF_PARAM_GRID, feat_df.iloc[train], y[train])
cs_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
#cs_model = deepcopy(asas_model.best_estimator_)
cs_model.fit(fset.iloc[train], y[train])

In [ ]:
cs_pred = cs_model.predict(fset)
print(f"Training score: {100 * np.mean(cs_pred[train] == y[train])}%")
print(f"Validation score: {100 * np.mean(cs_pred[valid] == y[valid])}%")
plot_confusion_matrix(y[valid], cs_model.predict(fset.iloc[valid]), classnames,
                      'figures/asas_richards_confusion.pdf' if SAVE_PLOTS else None)

## LINEAR reconstructions

In [ ]:
import os
import joblib
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from light_curve import LightCurve
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras_util import parse_model_args, get_run_id
from survey_autoencoder import main as survey_autoencoder

arg_dict = {'size': 96, 'embedding': 64, 'num_layers': 2, 'model_type': 'gru',
            'sim_type': 'linear/n200', 'n_min': 200, 'n_max': 200,
            'lr': 1e-3, 'bidirectional': True, 'drop_frac': 0.25, 'period_fold': False,
            'survey_files': ['data/linear/full.pkl'], 'no_train': True}
run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X, X_raw, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)

In [ ]:
(len(X), 0.8 * len(X), 0.2 * len(X))

In [ ]:
pred  = model.predict({'main_input': X[:], 'aux_input': X[:, :, 0:1]})
ords = np.argsort(np.mean((pred.squeeze() - X[:, :, 1]) ** 2 / (X_raw[:, :, 2] / scales), axis=1))
inds, labels = zip(*[(1142, '25'), (3425, '75')])
#for i, label in zip(inds, labels):
#    t = X_raw[~wrong_units][i, :, 0]
#    m = X_raw[~wrong_units][i, :, 1]
#    e = X_raw[~wrong_units][i, :, 2]
#    m_est = pred[i] * scales[i, 0] + means[i]
#    plt.figure()
#    plt.errorbar(t, m, e, None, 'o');
#    plt.errorbar(t, m_est, None, None, 'o');
#    plt.xlabel('Time [day]')
#    plt.ylabel('Magnitude')
#    plt.legend(['Original', 'Reconstructed'])
#    plt.gca().invert_yaxis()
#    if SAVE_PLOTS:
#        plt.savefig(f'figures/linear_reconstruct_{label}.pdf')

In [ ]:
%%time
import os
import joblib
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder

arg_dict = {'size': 96, 'embedding': 64, 'num_layers': 2, 'model_type': 'gru',
            'sim_type': 'asas_linear_fold/n200_ss0.7', 'n_min': 200, 'n_max': 200,
            'lr': 5e-4, 'bidirectional': True,
            'drop_frac': 0.25, 'survey_files': ['data/linear/full.pkl'],
            'period_fold': True, 'no_train': True}
run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X, X_raw, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)
pred_fold = model.predict({'main_input': X[:], 'aux_input': X[:, :, [0,]]}, batch_size=2000)
ords_fold = np.argsort(np.sum((pred_fold.squeeze() - X[:, :, 1]) ** 2, axis=1))

In [ ]:
full = joblib.load('data/linear/full.pkl')
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
for ord_i, label in zip(inds, labels):
    i = ords[ord_i]
    print(i)
    t = X_raw[~wrong_units][i, :, 0]
    m = X_raw[~wrong_units][i, :, 1]
    e = X_raw[~wrong_units][i, :, 2]
    m_est = (pred[i] * scales[i, 0] + means[i])[np.argsort(split[i].times % split[i].p)]
    m_fold = pred_fold[i] * scales[i, 0] + means[i]
    plt.figure()
    plt.errorbar(t, m, e, None, 'o');
    plt.errorbar(t, m_est, None, None, 'o', alpha=0.6);
    plt.errorbar(t, m_fold, None, None, 'o');
    plt.xlabel('Phase [day]')
    plt.ylabel('Magnitude')
    plt.legend(['Original', 'Reconstructed (non-folded)', 'Reconstructed (folded)'])
    plt.title(f'{split[i].survey} {split[i].name} ({split[i].label})')
    plt.gca().invert_yaxis()
    if SAVE_PLOTS:
        plt.savefig(f'figures/linear_folded_reconstruct_{label}.pdf')

## LINEAR autoencoder random forests

In [ ]:
%%time
import os
import joblib
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from light_curve import LightCurve
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder, preprocess

#arg_dict = {'size': 96, 'embedding': 64, 'num_layers': 2, 'model_type': 'gru',
#        'sim_type': 'asas_linear_fold/n200_ss0.7', 'n_min': 200, 'n_max': 200,
#        'lr': 5e-4, 'bidirectional': True, 'drop_frac': 0.25, 'period_fold': True,
#        'survey_files': ['data/linear/full.pkl'], 'no_train': True}
arg_dict = args.__dict__
run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X, X_raw, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)

full = joblib.load('data/linear/full.pkl')
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
if args.period_fold:
    for lc in split:
        lc.period_fold()
X_list = [np.c_[lc.times, lc.measurements, lc.errors] for lc in split]
classnames, indices = np.unique([lc.label for lc in split], return_inverse=True)
periods = [lc.p for lc in split]
y = classnames[indices]
#train, valid = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y))[0]

encode_model = Model(input=model.input, output=model.get_layer('encoding').output)
encoding = encode_model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)})
encoding = np.c_[encoding, means, scales, periods]
#linear_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
#linear_model.fit(encoding[train], y[train])
#print("Training score: {}%".format(100 * linear_model.score(encoding[train], y[train])))
#print("Validation score: {}%".format(100 * linear_model.score(encoding[valid], y[valid])))
###print(f"OOB score: {100 * oob_score}")
##plot_confusion_matrix(y[valid], cs_model.predict(encoding[valid]), classnames,
##                      'figures/linear_confusion.pdf' if SAVE_PLOTS else None)

In [ ]:
print("LINEAR autoencoder")
for train, valid in StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y):
    linear_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
    linear_model.fit(encoding[train], y[train])
    print("Training score: {}%".format(100 * linear_model.score(encoding[train], y[train])))
    print("Validation score: {}%".format(100 * linear_model.score(encoding[valid], y[valid])))

## LINEAR Richards random forests

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold
from light_curve import LightCurve
import joblib
from cesium.features import LOMB_SCARGLE_FEATS, CADENCE_FEATS, GENERAL_FEATS
from cesium.featurize import featurize_time_series

from argparse import Namespace; args = Namespace(n_min=200, n_max=200)

full = joblib.load('data/linear/full.pkl')
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
X_list = [np.c_[lc.times, lc.measurements, lc.errors] for lc in split]
classnames, indices = np.unique([lc.label for lc in split], return_inverse=True)
y = classnames[indices]
train, valid = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y))[0]

times, measurements, errors = zip(*[x.T for x in X_list])

features_to_use = LOMB_SCARGLE_FEATS + GENERAL_FEATS# + CADENCE_FEATS

#fset = featurize_time_series(times, measurements, errors, features_to_use)
#joblib.dump(fset, 'linear_richards.pkl', compress=True)
fset = joblib.load('linear_richards.pkl')

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

cs_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
#cs_model = deepcopy(asas_model.best_estimator_)
cs_model.fit(fset.iloc[train], y[train])

In [ ]:
cs_pred = cs_model.predict(fset)
print(f"Training score: {100 * np.mean(cs_pred[train] == y[train])}%")
print(f"Validation score: {100 * np.mean(cs_pred[valid] == y[valid])}%")
plot_confusion_matrix(y[valid], cs_model.predict(fset.iloc[valid]), classnames,
                      'figures/linear_richards_confusion.pdf' if SAVE_PLOTS else None)

## LINEAR noisy examples

In [ ]:
%run survey_autoencoder.py --size 96 --num_layers 2 --drop_frac 0.25 --no_train \
                           --model_type gru --lr 1e-3 --sim_type linear/n200 \
                           --n_min 200 --n_max 200 --embedding 64 \
                           --survey_files data/linear/full.pkl --bidirectional --period_fold
            
full = joblib.load('data/linear/full.pkl')
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
if args.period_fold:
    for lc in split:
        lc.period_fold()

#lc = [lc for lc in split if lc.name == '3182815'][0]
lc = [lc for lc in split if lc.name == '1018967'][0]

X = np.c_[lc.times, lc.measurements, lc.errors][np.newaxis, :]
errors = X[:, :, 2]
sample_gen = ku.noisify_samples({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)},
                                X[:, :, [1]], errors, batch_size=1, sample_weight=None)

In [ ]:
colors = sns.color_palette('deep', n_colors=2)
t = X_raw[0, :, 0]
m = X[0, :, 1]
e = errors[0]
plt.figure()
sns.set_style("darkgrid")
plt.errorbar(t, m, e, None, 'o', color=colors[0]);
for i in range(3):
    (X_i, _), y_i, w_i = next(sample_gen)
    plt.plot(t, X[:, :, 1].mean() + X[:, :, 1].std() * X_i[0, :, 1], 'o', alpha=0.5, color=colors[1])
    if i == 0:
        plt.legend(['Sampled', 'Original'])
plt.xlabel('Time [day]')
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()
plt.title(f'{lc.survey} {lc.name} ({lc.label})')

if 1:
    plt.savefig('paper/figures/linear_noisy.pdf')

## Transfer learning

In [ ]:
%%time
# LINEAR autoencoder -> ASAS RF -> ASAS
import os
import joblib
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from light_curve import LightCurve
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder, preprocess

arg_dict = {'size': 96, 'embedding': 64, 'num_layers': 2, 'model_type': 'gru',
            'sim_type': 'linear_fold/n200', 'n_min': 200, 'n_max': 200,
            'lr': 1e-3, 'bidirectional': True,# 'ss_resid': 0.7,
            'drop_frac': 0.25, 'survey_files': ['data/linear/full.pkl'],
            'period_fold': True, 'no_train': True}
run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X, X_raw, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)

top_classes = ['RR_Lyrae_FM', 'W_Ursae_Maj', 'Classical_Cepheid', 'Beta_Persei', 'Semireg_PV']
full = joblib.load('data/asas/full.pkl')
full = [lc for lc in full if lc.label in top_classes]
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
#if args.ss_resid:
#    split = [el for el in split if el.ss_resid <= args.ss_resid]
if args.period_fold:
    for lc in split:
        lc.period_fold()
X_list = [np.c_[lc.times, lc.measurements, lc.errors] for lc in split]
classnames, indices = np.unique([lc.label for lc in split], return_inverse=True)
periods = [lc.p for lc in split]
y = classnames[indices]
train, valid = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y))[0]

X_raw = pad_sequences(X_list, value=0., dtype='float', padding='post')
X, means, scales, wrong_units = preprocess(X_raw, args.m_max)
encode_model = Model(input=model.input, output=model.get_layer('encoding').output)
encoding = encode_model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)})
encoding = np.c_[encoding, means, scales, periods]

asas_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
asas_model.fit(encoding[train], y[train])
print("Training score: {}%".format(100 * asas_model.score(encoding[train], y[train])))
print("Validation score: {}%".format(100 * asas_model.score(encoding[valid], y[valid])))

In [ ]:
%%time
# ASAS autoencoder -> LINEAR RF -> LINEAR
import os
import joblib
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from light_curve import LightCurve
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder, preprocess

arg_dict = {'size': 64, 'embedding': 64, 'num_layers': 1, 'model_type': 'gru',
            'sim_type': 'asas_fold/n200_ss0.7', 'lomb_score': None, 'n_min': 200, 'n_max': 200,
            'lr': 1e-3, 'bidirectional': True, 'drop_frac': 0.25, 'period_fold': True,
            'ss_resid': 0.7, 'survey_files': ['data/asas/full.pkl'], 'no_train': True}
run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X, X_raw, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)

full = joblib.load('data/linear/full.pkl')
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
if args.period_fold:
    for lc in split:
        lc.period_fold()
X_list = [np.c_[lc.times, lc.measurements, lc.errors] for lc in split]
classnames, indices = np.unique([lc.label for lc in split], return_inverse=True)
periods = [lc.p for lc in split]
y = classnames[indices]
train, valid = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y))[0]

X_raw = pad_sequences(X_list, value=0., dtype='float', padding='post')
X, means, scales, wrong_units = preprocess(X_raw, args.m_max)
encode_model = Model(input=model.input, output=model.get_layer('encoding').output)
encoding = encode_model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)})
encoding = np.c_[encoding, means, scales, periods]
linear_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
linear_model.fit(encoding[train], y[train])
print("Training score: {}%".format(100 * linear_model.score(encoding[train], y[train])))
print("Validation score: {}%".format(100 * linear_model.score(encoding[valid], y[valid])))

In [ ]:
#%%time
import os
import joblib
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from light_curve import LightCurve
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder, preprocess

arg_dict = {'size': 96, 'embedding': 64, 'num_layers': 2, 'model_type': 'gru',
            'lomb_score': None, 'n_min': 200, 'n_max': 200,
            'lr': 5e-4, 'bidirectional': True, 'drop_frac': 0.25, 'period_fold': True,
            'no_train': True}

def load_and_encode(arg_dict):
    run = get_run_id(**arg_dict)
    log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
    weights_path = os.path.join(log_dir, 'weights.h5')
    if not os.path.exists(weights_path):
        raise FileNotFoundError(weights_path)
    _, _, model, _, _, _, args = survey_autoencoder(arg_dict)

    full = joblib.load(arg_dict['survey_files'][0])
    if 'asas' in arg_dict['survey_files'][0]:
        top_classes = ['RR_Lyrae_FM', 'W_Ursae_Maj', 'Classical_Cepheid', 'Beta_Persei', 'Semireg_PV']
        full = [lc for lc in full if lc.label in top_classes]
    split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
    if args.period_fold:
        for lc in split:
            lc.period_fold()
    X_list = [np.c_[lc.times, lc.measurements, lc.errors] for lc in split]
    classnames, indices = np.unique([lc.label for lc in split], return_inverse=True)
    periods = [lc.p for lc in split]
    y = classnames[indices]
    train, valid = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y))[0]

    X_raw = pad_sequences(X_list, value=0., dtype='float', padding='post')
    X, means, scales, wrong_units = preprocess(X_raw, args.m_max)
    encode_model = Model(input=model.input, output=model.get_layer('encoding').output)
    encoding = encode_model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)})
    encoding = np.c_[encoding, means, scales, periods]
    
    return encoding, y

asas_encoding_asas, y_asas = load_and_encode({**arg_dict, 'sim_type': 'asas_fold/n200_ss0.7',
                                              'survey_files': ['data/asas/full.pkl']})
asas_encoding_linear, y_linear = load_and_encode({**arg_dict, 'sim_type': 'asas_fold/n200_ss0.7',
                                                  'survey_files': ['data/linear/full.pkl']})
linear_encoding_asas, _ = load_and_encode({**arg_dict, 'sim_type': 'linear_fold/n200',
                                           'survey_files': ['data/asas/full.pkl']})
linear_encoding_linear, _ = load_and_encode({**arg_dict, 'sim_type': 'linear_fold/n200',
                                             'survey_files': ['data/linear/full.pkl']})

fset_asas = joblib.load('asas_richards.pkl')
fset_linear = joblib.load('linear_richards.pkl')

In [ ]:
splits_asas = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(y_asas, y_asas)
splits_linear = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(y_linear, y_linear)

feature_dict = {
    'asas_auto': (asas_encoding_asas, asas_encoding_linear),
    'linear_auto': (linear_encoding_asas, linear_encoding_linear),
    'richards': (fset_asas, fset_linear)
}

from collections import defaultdict
asas_scores = defaultdict(list)    # key[0] features, trained on key[1], scored on y_asas
linear_scores = defaultdict(list)  # key[0] features, trained on key[1], scored on y_linear
for label, (X_asas, X_linear) in feature_dict.items():
    for train, valid in splits_asas:
        import datetime; print(label, datetime.datetime.now())
        asas_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
        asas_model.fit(X_asas[train], y_asas[train])
        asas_scores[(label, 'asas')].append(asas_model.score(X_asas[valid], y_asas[valid]))

    for train, valid in splits_linear:
        import datetime; print(label, datetime.datetime.now())
        linear_model = GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID)
        linear_model.fit(X_linear[train], y_linear[train])
        linear_scores[(label, 'linear')].append(linear_model.score(X_linear[valid], y_linear[valid]))
        
    asas_model.fit(X_asas, y_asas)
    linear_scores[(label, 'asas')] = [asas_model.score(X_linear, y_linear)]

    linear_model.fit(X_linear, y_linear)
    asas_scores[(label, 'linear')] = [linear_model.score(X_asas, y_asas)]
    break  # TODO temporary

In [ ]:
asas_scores

## Dropout plots 

In [ ]:
log_files = glob.glob('keras_logs/linear_fold_valid/n200/gru_096_x2_5m04*emb64_bidir/training.csv')

step_logs, time_logs = parse_logs(log_files)
step_plot = training_plot(step_logs, ylim=(0.5, 10.0))
step_plot.axes[0].legend(labels=[f"Dropout={d}%" for d in [0, 12.5, 25, 37.5]]);
time_plot = training_plot(time_logs, ylim=(0.5, 10.0))
time_plot.axes[0].legend(labels=[f"Dropout={d}%" for d in [0, 12.5, 25, 37.5]]);
if 1:
    step_plot.savefig('paper/figures/linear_dropout_step.pdf')
    time_plot.savefig('paper/figures/linear_dropout_time.pdf')

## Variable length plots

In [ ]:
log_files = [f'keras_logs/linear_fold_valid/n{i}/gru_096_x2_5m04_drop25_emb64_bidir/training.csv'
             for i in ['200', '150_250', '100_300']]

step_logs, time_logs = parse_logs(log_files, 2000)
step_plot = training_plot(step_logs, ylim=(0.5, 15.0))
step_plot.axes[0].legend(labels=["N=200", "150≤N≤250", "100≤N≤300"])
time_plot = training_plot(time_logs, ylim=(0.5, 15.0))
time_plot.axes[0].legend(labels=["N=200", "150≤N≤250", "100≤N≤300"])
if 1:
    step_plot.savefig('paper/figures/linear_var_length_step.pdf')
    time_plot.savefig('paper/figures/linear_var_length_time.pdf')

In [ ]:
df = pd.concat((pd.DataFrame({'a': [1.0, 2.0, 3.0]}), pd.DataFrame({'a': [4.0, 5.0, 6.0]})),
               axis=1)
print("Before:\n", df)
c = df.iloc[:, 0]
c.dropna(inplace=True)
print("After:\n", df)

## MACHO reconstructions

In [ ]:
%%time
import os
from light_curve import LightCurve
import joblib
from keras_util import get_run_id
from survey_autoencoder import main as survey_autoencoder

arg_dict = {'size': 96, 'embedding': 64, 'num_layers': 2, 'model_type': 'gru',
            'sim_type': 'macho_fold/n200_ss0.7_long', 'n_min': 200, 'n_max': 200,
            'lr': 5e-4, 'bidirectional': True, 'ss_resid': 0.7,
            'drop_frac': 0.25, 'survey_files': ['data/macho/full.pkl'],
            'period_fold': True, 'no_train': True}

run = get_run_id(**arg_dict)
log_dir = os.path.join(os.getcwd(), 'keras_logs', arg_dict['sim_type'], run)
weights_path = os.path.join(log_dir, 'weights.h5')
if not os.path.exists(weights_path):
    raise FileNotFoundError(weights_path)
X, X_raw, model, means, scales, wrong_units, args = survey_autoencoder(arg_dict)
full = joblib.load('data/macho/full.pkl')
split = [el for lc in full for el in lc.split(args.n_min, args.n_max)]
#if args.ss_resid:
#    split = [el for el in split if el.ss_resid <= args.ss_resid]

In [ ]:
X.shape[0] * np.array([0.25, 0.75])

In [ ]:
%%time
pred = model.predict({'main_input': X[:], 'aux_input': X[:, :, [0,]]})
ords = np.argsort(np.sum((pred.squeeze() - X[:, :, 1]) ** 2, axis=1))
inds, labels = zip(*[(5195, '25'), (15585, '75')])
for ord_i, label in zip(inds, labels):
    i = ords[ord_i]
    print(i)
    t = X_raw[~wrong_units][i, :, 0]
    m = X_raw[~wrong_units][i, :, 1]
    e = X_raw[~wrong_units][i, :, 2]
    m_est = pred[i] * scales[i, 0] + means[i]
    plt.figure()
    plt.errorbar(t, m, e, None, 'o');
    plt.errorbar(t, m_est, None, None, 'o');
    plt.xlabel('Time [day]')
    plt.ylabel('Magnitude')
    plt.legend(['Original', 'Reconstructed'])
    plt.title(f'{split[i].survey} {split[i].name} ({split[i].label})')
    plt.gca().invert_yaxis()
    if SAVE_PLOTS:
        plt.savefig(f'paper/figures/macho_reconstruct_{label}.pdf')

In [ ]:
%%time
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from keras.preprocessing.sequence import pad_sequences
from survey_autoencoder import main as survey_autoencoder, preprocess
from keras.models import Model
from sklearn.ensemble import RandomForestClassifier

for lc in split:
    lc.period_fold()
X_list = [np.c_[lc.times, lc.measurements, lc.errors] for lc in split]
classnames, indices = np.unique([lc.label for lc in split], return_inverse=True)
periods = [lc.p for lc in split]
y = classnames[indices]
train, valid = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED).split(X_list, y))[0]

X_raw = pad_sequences(X_list, value=0., dtype='float', padding='post')
X, means, scales, wrong_units = preprocess(X_raw, args.m_max)
encode_model = Model(input=model.input, output=model.get_layer('encoding').output)
encoding = encode_model.predict({'main_input': X, 'aux_input': np.delete(X, 1, axis=2)})
encoding = np.c_[encoding, means, scales, periods]

In [ ]:
(len(full), len(split))

In [ ]:
encoding.shape

In [ ]:
train.shape

In [ ]:
#from distributed import Client
import dask.multiprocessing
import dask_searchcv as dcv

train = np.arange(500)

macho_model = dcv.GridSearchCV(RandomForestClassifier(random_state=0), RF_PARAM_GRID,
                               scheduler=dask.multiprocessing.get)
%time macho_model.fit(encoding[train], y[train])